In [1]:
import os
import numpy as np
from argparse import ArgumentParser
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import BertTokenizer, TFBertForSequenceClassification, AutoConfig
from dataclasses import asdict
from transformers.data.processors.utils import DataProcessor, InputExample, InputFeatures

print("슝~")

슝~


In [2]:
!pip install tensorflow-datasets -U

     |████████████████████████████████| 4.0 MB 4.7 MB/s 
  Attempting uninstall: tensorflow-datasets
    Found existing installation: tensorflow-datasets 4.2.0
    Uninstalling tensorflow-datasets-4.2.0:
      Successfully uninstalled tensorflow-datasets-4.2.0
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
data, info = tfds.load('glue/mnli', with_info=True)
info.splits['train'].num_examples

INFO:absl:Load pre-computed DatasetInfo (eg: splits, num examples,...) from GCS: glue/mnli/2.0.0
INFO:absl:Load dataset info from /tmp/tmp9ngcr9r6tfds
INFO:absl:Generating dataset glue (/aiffel/tensorflow_datasets/glue/mnli/2.0.0)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

INFO:absl:Downloading https://dl.fbaipublicfiles.com/glue/data/MNLI.zip into /aiffel/tensorflow_datasets/downloads/dl.fbaipublicfiles.com_glue_MNLIdNe8cK2kTBCG0bqBz2JxwShRT2KfuO3NVIwROTnjtfI.zip.tmp.37d1eb7e3c9c4d3f8180e8bd3b2af062...


...

Generating test_mismatched examples...:   0%|          | 0/9847 [00:00<?, ? examples/s]

Shuffling glue-test_mismatched.tfrecord...:   0%|          | 0/9847 [00:00<?, ? examples/s]

INFO:absl:Done writing glue-test_mismatched.tfrecord. Number of examples: 9847 (shards: [9847])
INFO:absl:Constructing tf.data.Dataset glue for split None, from /aiffel/tensorflow_datasets/glue/mnli/2.0.0


Dataset glue downloaded and prepared to /aiffel/tensorflow_datasets/glue/mnli/2.0.0. Subsequent calls will reuse this data.


392702

In [3]:
data['train'].take(1)

<TakeDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>

In [5]:
examples = data['train'].take(1)
for example in examples:
    hypothesis = example['hypothesis']
    label = example['label']
    print(hypothesis)
    print(label)

tf.Tensor(b'Meaningful partnerships with stakeholders is crucial.', shape=(), dtype=string)
tf.Tensor(1, shape=(), dtype=int64)


In [6]:
class DataProcessor:
    """Base class for data converters for sequence classification data sets."""

    def get_example_from_tensor_dict(self, tensor_dict):
        """
        Gets an example from a dict with tensorflow tensors.

        Args:
            tensor_dict: Keys and values should match the corresponding Glue
                tensorflow_dataset examples.
        """
        raise NotImplementedError()

    def get_train_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the dev set."""
        raise NotImplementedError()

    def get_test_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the test set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    def tfds_map(self, example):
        """
        Some tensorflow_datasets datasets are not formatted the same way the GLUE datasets are. This method converts
        examples to the correct format.
        """
        if len(self.get_labels()) > 1:
            example.label = self.get_labels()[int(example.label)]
        return example

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8-sig") as f:
            return list(csv.reader(f, delimiter="\t", quotechar=quotechar))

print("슝~")

슝~


In [7]:

class MnliProcessor(DataProcessor):
    """Processor for the MultiNLI data set (GLUE version)."""

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        #warnings.warn(DEPRECATION_WARNING.format("processor"), FutureWarning)

    def get_example_from_tensor_dict(self, tensor_dict):
        """See base class."""
        return InputExample(
            tensor_dict["idx"].numpy(),
            tensor_dict["premise"].numpy().decode("utf-8"),
            tensor_dict["hypothesis"].numpy().decode("utf-8"),
            str(tensor_dict["label"].numpy()),
        )

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "dev_matched.tsv")), "dev_matched")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "test_matched.tsv")), "test_matched")

    def get_labels(self):
        """See base class."""
        return ["contradiction", "entailment", "neutral"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training, dev and test sets."""
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = f"{set_type}-{line[0]}"
            text_a = line[8]
            text_b = line[9]
            label = None if set_type.startswith("test") else line[-1]
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples


In [8]:
processor = MnliProcessor()
examples = data['train'].take(1)

for example in examples:
    print('------원본데이터------')
    print(example)  
    example = processor.get_example_from_tensor_dict(example)
    print('------processor 가공데이터------')
    print(example)

------원본데이터------
{'hypothesis': <tf.Tensor: shape=(), dtype=string, numpy=b'Meaningful partnerships with stakeholders is crucial.'>, 'idx': <tf.Tensor: shape=(), dtype=int32, numpy=16399>, 'label': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'premise': <tf.Tensor: shape=(), dtype=string, numpy=b'In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.'>}
------processor 가공데이터------
InputExample(guid=16399, text_a='In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.', text_b='Meaningful partnerships with stakeh

In [9]:
examples = (data['train'].take(1))
for example in examples:
    example = processor.get_example_from_tensor_dict(example)
    example = processor.tfds_map(example)
    print(example)

InputExample(guid=16399, text_a='In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.', text_b='Meaningful partnerships with stakeholders is crucial.', label='entailment')


In [10]:
label_list = processor.get_labels()
label_list

['contradiction', 'entailment', 'neutral']

In [11]:
label_map = {label: i for i, label in enumerate(label_list)}
label_map

{'contradiction': 0, 'entailment': 1, 'neutral': 2}

In [21]:
from transformers import GPT2Tokenizer, TFGPT2Model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', pad_token='[PAD]')
model = TFGPT2Model.from_pretrained('gpt2')
text = "hihi"
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.
All model checkpoint layers were used when initializing TFGPT2Model.

All the layers of TFGPT2Model were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


In [13]:
def _glue_convert_examples_to_features(examples, tokenizer, max_length, processor, label_list=None, output_mode="claasification") :
    if max_length is None :
        max_length = tokenizer.max_len
    if label_list is None:
        label_list = processor.get_labels()
        print("Using label list %s" % (label_list))

    label_map = {label: i for i, label in enumerate(label_list)}
    labels = [label_map[example.label] for example in examples]

    batch_encoding = tokenizer(
        [(example.text_a, example.text_b) for example in examples],
        max_length=max_length,
        padding="max_length",
        truncation=True,
    )

    features = []
    for i in range(len(examples)):
        inputs = {k: batch_encoding[k][i] for k in batch_encoding}

        feature = InputFeatures(**inputs, label=labels[i])
        features.append(feature)

    for i, example in enumerate(examples[:5]):
        print("*** Example ***")
        print("guid: %s" % (example.guid))
        print("features: %s" % features[i])

    return features

print("슝~")

슝~


In [14]:
def tf_glue_convert_examples_to_features(examples, tokenizer, max_length, processor, label_list=None, output_mode="classification") :
    """
    :param examples: tf.data.Dataset
    :param tokenizer: pretrained tokenizer
    :param max_length: example의 최대 길이(기본값 : tokenizer의 max_len)
    :param task: GLUE task 이름
    :param label_list: 라벨 리스트
    :param output_mode: "regression" or "classification"

    :return: task에 맞도록 feature가 구성된 tf.data.Dataset
    """
    examples = [processor.tfds_map(processor.get_example_from_tensor_dict(example)) for example in examples]
    features = _glue_convert_examples_to_features(examples, tokenizer, max_length, processor)
    label_type = tf.int64

    def gen():
        for ex in features:
            d = {k: v for k, v in asdict(ex).items() if v is not None}
            label = d.pop("label")
            yield (d, label)

    input_names = ["input_ids"] + tokenizer.model_input_names

    return tf.data.Dataset.from_generator(
        gen,
        ({k: tf.int32 for k in input_names}, label_type),
        ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([])),
    )

print("슝~")

슝~


In [22]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token':'[PAD]'})

In [23]:
train_dataset = tf_glue_convert_examples_to_features(data['train'], tokenizer, max_length=128, processor=processor)

Using label list ['contradiction', 'entailment', 'neutral']
*** Example ***
guid: 16399
features: InputFeatures(input_ids=[818, 9465, 286, 777, 15733, 11, 406, 6173, 468, 3111, 44243, 1201, 8735, 284, 13878, 262, 9027, 286, 262, 1812, 21913, 18362, 290, 284, 4474, 11570, 22867, 351, 26138, 8998, 379, 44209, 257, 649, 5307, 42960, 1022, 262, 2717, 10131, 290, 20352, 286, 2742, 2594, 4918, 13, 5308, 7574, 913, 22867, 351, 26138, 318, 8780, 13, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
examples = train_dataset.take(1)
for example in examples:
    print(example)

({'input_ids': <tf.Tensor: shape=(128,), dtype=int32, numpy=
array([  818,  9465,   286,   777, 15733,    11,   406,  6173,   468,
        3111, 44243,  1201,  8735,   284, 13878,   262,  9027,   286,
         262,  1812, 21913, 18362,   290,   284,  4474, 11570, 22867,
         351, 26138,  8998,   379, 44209,   257,   649,  5307, 42960,
        1022,   262,  2717, 10131,   290, 20352,   286,  2742,  2594,
        4918,    13,  5308,  7574,   913, 22867,   351, 26138,   318,
        8780,    13, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
       50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
       50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
       50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
       50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
       50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
       50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
       50257, 50257, 50257, 5

In [25]:
# train 데이터셋
train_dataset = tf_glue_convert_examples_to_features(data['train'], tokenizer, max_length=128, processor=processor)
train_dataset_batch = train_dataset.shuffle(100).batch(16).repeat(2)

Using label list ['contradiction', 'entailment', 'neutral']
*** Example ***
guid: 16399
features: InputFeatures(input_ids=[818, 9465, 286, 777, 15733, 11, 406, 6173, 468, 3111, 44243, 1201, 8735, 284, 13878, 262, 9027, 286, 262, 1812, 21913, 18362, 290, 284, 4474, 11570, 22867, 351, 26138, 8998, 379, 44209, 257, 649, 5307, 42960, 1022, 262, 2717, 10131, 290, 20352, 286, 2742, 2594, 4918, 13, 5308, 7574, 913, 22867, 351, 26138, 318, 8780, 13, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [29]:
data.keys()

dict_keys([Split('train'), 'validation_matched', 'validation_mismatched', 'test_matched', 'test_mismatched'])

In [31]:
# validation_matched 데이터셋
validation_matched_dataset = tf_glue_convert_examples_to_features(data['validation_matched'], tokenizer, max_length=128, processor=processor)
validation_matched_dataset_batch = validation_matched_dataset.shuffle(100).batch(16)

Using label list ['contradiction', 'entailment', 'neutral']
*** Example ***
guid: 6287
features: InputFeatures(input_ids=[7456, 12, 71, 7456, 11752, 10194, 477, 262, 661, 329, 826, 21480, 1204, 393, 1223, 43669, 6041, 286, 661, 329, 262, 826, 1204, 220, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257], attention_mask=[1

In [34]:
# validation_mismatched 데이터셋
validation_mismatched_dataset = tf_glue_convert_examples_to_features(data['validation_mismatched'], tokenizer, max_length=128, processor=processor)
validation_mismatched_dataset_batch = validation_mismatched_dataset.shuffle(100).batch(16)

Using label list ['contradiction', 'entailment', 'neutral']
*** Example ***
guid: 9410
features: InputFeatures(input_ids=[16775, 82, 543, 551, 75, 1469, 290, 22465, 262, 3710, 1998, 290, 3197, 617, 286, 674, 18822, 9123, 290, 7799, 284, 674, 7611, 438, 392, 284, 674, 1748, 13, 4711, 4493, 389, 5688, 9514, 290, 836, 470, 2928, 2687, 13, 220, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [35]:
# test_matched 데이터셋
test_matched_dataset = tf_glue_convert_examples_to_features(data['test_matched'], tokenizer, max_length=128, processor=processor)
test_matched_dataset_batch = test_matched_dataset.shuffle(100).batch(16)

Using label list ['contradiction', 'entailment', 'neutral']
*** Example ***
guid: 5398
features: InputFeatures(input_ids=[4053, 318, 612, 257, 1310, 5186, 8901, 345, 1234, 287, 612, 3792, 612, 257, 8901, 326, 345, 1234, 287, 612, 30, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257], attention_mask=[

In [36]:
# test_mismatched 데이터셋
test_mismatched_dataset = tf_glue_convert_examples_to_features(data['test_mismatched'], tokenizer, max_length=128, processor=processor)
test_mismatched_dataset_batch = test_mismatched_dataset.shuffle(100).batch(16)

Using label list ['contradiction', 'entailment', 'neutral']
*** Example ***
guid: 3498
features: InputFeatures(input_ids=[23795, 11, 314, 14068, 326, 3084, 13, 2504, 3084, 468, 587, 287, 616, 1641, 329, 10439, 13, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257], attentio

In [37]:
# TFTrainer을 활용하는 형태로 모델 재생성
from transformers import (
    AutoConfig,
    TFGPT2Model,
    AutoTokenizer,
    EvalPrediction,
    HfArgumentParser,
    PreTrainedTokenizer,
    TFAutoModelForSequenceClassification,
    TFTrainer,
    TFTrainingArguments,
    glue_compute_metrics,
    glue_convert_examples_to_features,
    glue_output_modes,
    glue_processors,
    glue_tasks_num_labels,
)

output_dir = os.getenv('HOME')+'/aiffel/transformers/mnli'

training_args = TFTrainingArguments(
    output_dir=output_dir,            # output이 저장될 경로
    num_train_epochs=3,              # train 시킬 총 epochs
    per_device_train_batch_size=16,  # 각 device 당 batch size
    per_device_eval_batch_size=64,   # evaluation 시에 batch size
    warmup_steps=500,                # learning rate scheduler에 따른 warmup_step 설정
    weight_decay=0.01,                 # weight decay
    logging_dir='./mnli/logs',                 # log가 저장될 경로
    do_train=True,
    do_eval=True,
    eval_steps=1000
)

max_seq_length=128
task_name = "mnli"

with training_args.strategy.scope():
    model = TFGPT2Model.from_pretrained('gpt2')

All model checkpoint layers were used when initializing TFGPT2Model.

All the layers of TFGPT2Model were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


In [38]:
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    if output_mode == "classification":
        preds = np.argmax(p.predictions, axis=1)
    elif output_mode == "regression":
        preds = np.squeeze(p.predictions)
    return glue_compute_metrics(task_name, preds, p.label_ids)

output_mode = glue_output_modes[task_name]
output_mode

'classification'

In [39]:
train_dataset = train_dataset.apply(tf.data.experimental.assert_cardinality(info.splits['train'].num_examples))
validation_matched_dataset = validation_matched_dataset.apply(tf.data.experimental.assert_cardinality(info.splits['validation_matched'].num_examples))
validation_mismatched_dataset = validation_mismatched_dataset.apply(tf.data.experimental.assert_cardinality(info.splits['validation_mismatched'].num_examples))
test_matched_dataset = test_matched_dataset.apply(tf.data.experimental.assert_cardinality(info.splits['test_matched'].num_examples))
test_mismatched_dataset = test_mismatched_dataset.apply(tf.data.experimental.assert_cardinality(info.splits['test_mismatched'].num_examples))

In [40]:
trainer = TFTrainer(
    model=model,                           # 학습시킬 model
    args=training_args,                  # TFTrainingArguments을 통해 설정한 arguments
    train_dataset=train_dataset,    # training dataset
    eval_dataset=validation_matched_dataset,       # evaluation dataset
    compute_metrics=compute_metrics,
)


In [41]:
# Training
if training_args.do_train:
    trainer.train()
    trainer.save_model()
    tokenizer.save_pretrained(training_args.output_dir)


ValueError: in user code:

    /opt/conda/lib/python3.7/site-packages/transformers/trainer_tf.py:672 distributed_training_steps  *
        self.args.strategy.run(self.apply_gradients, inputs)
    /opt/conda/lib/python3.7/site-packages/transformers/trainer_tf.py:633 apply_gradients  *
        gradients = self.training_step(features, labels, nb_instances_in_global_batch)
    /opt/conda/lib/python3.7/site-packages/transformers/trainer_tf.py:616 training_step  *
        per_example_loss, _ = self.run_model(features, labels, True)
    /opt/conda/lib/python3.7/site-packages/transformers/trainer_tf.py:719 run_model  *
        outputs = self.model(features, labels=labels, training=training)[:2]
    /opt/conda/lib/python3.7/site-packages/transformers/models/gpt2/modeling_tf_gpt2.py:599 call  *
        inputs = input_processing(
    /opt/conda/lib/python3.7/site-packages/transformers/modeling_tf_utils.py:345 input_processing  *
        raise ValueError(

    ValueError: The following keyword arg

In [ ]:
import numpy as np

# Evaluation
results = {}
if training_args.do_eval:
    result = trainer.evaluate()
    output_eval_file = os.path.join(training_args.output_dir, "eval_results.txt")

    with open(output_eval_file, "w") as writer:
        for key, value in result.items():
            writer.write(f"{key} = {value}\n")

        results.update(result)
        
#파일에 쓴 테스트 결과 확인
!cat ~/aiffel/transformers/eval_results2.txt